In [1]:
# Get packages
import numpy as np
import pandas as pd

In [2]:
# Import data
df1 = pd.read_csv('Annual_Parking_Study_Data.csv',low_memory=False)

In [3]:
def printna():
    cols = df1.columns
    for col in cols:
        num_nans = df1[col].isna().sum()
        print(col + ": " + str(num_nans))
        
printna()

Elmntkey: 78
Study_Area: 26
Sub_Area: 13180
Date Time: 0
Side: 39
Unitdesc: 26
Peak Hour_SDOT: 150574
Parking_Spaces: 298
Total_Vehicle_Count: 485
Dp_Count: 42687
Rpz_Count: 131222
TG_Car2Go: 131437
BMW_DN: 152367
Lime: 165552
Idling: 163714
Field Notes: 151822
Construction: 13
Event Closure: 13
Subarea Label: 0
Study Year: 26
Peak Hour? (Yes or No): 0
RPZ Blocks: 78103
CSM: 77895
Time Stamp: 83590


In [4]:
# Drop rows where Unitdesc (the street name) is missing
df1.dropna(axis = 0,subset=['Unitdesc'],inplace=True)

In [5]:
printna()

Elmntkey: 52
Study_Area: 0
Sub_Area: 13180
Date Time: 0
Side: 13
Unitdesc: 0
Peak Hour_SDOT: 150548
Parking_Spaces: 272
Total_Vehicle_Count: 459
Dp_Count: 42661
Rpz_Count: 131196
TG_Car2Go: 131411
BMW_DN: 152341
Lime: 165526
Idling: 163688
Field Notes: 151796
Construction: 13
Event Closure: 13
Subarea Label: 0
Study Year: 0
Peak Hour? (Yes or No): 0
RPZ Blocks: 78077
CSM: 77895
Time Stamp: 83564


In [6]:
# For those blocks missing a "Side" parameter, look up other entries for those blocks to see if we can infer which side it is
blocks_missing_side = df1['Unitdesc'][df1['Side'].isna()].unique()
for blk in blocks_missing_side:
    print(blk)
    this_block = df1[df1['Unitdesc']==blk]
    side_options = this_block['Side'].unique()
    #side_options = side_options[~np.isnan(side_options)]
    
    side_dict = {}
    for side in side_options:
        if not isinstance(side, str):
            print(side)
            continue
        else:
            this_side = this_block[this_block['Side']==side]
            side_dict[this_side['Elmntkey'].unique()[0]] = side
print(side_dict)

NE 41ST ST BETWEEN 11TH AVE NE AND 12TH AVE NE
nan
{39329.0: 'N', 39330.0: 'S'}


In [8]:
# Try to fill in the missing Side entries
df1.loc[df1['Side'].isna(),'Side'] = df1['Elmntkey'][df1['Side'].isna()].map(side_dict)

In [9]:
values = {'Peak Hour_SDOT':'None','Parking_Spaces':0,'Total_Vehicle_Count':0,'Dp_Count':0,'Rpz_Count':0,'TG_Car2Go':0,'BMW_DN':0,'Lime':0,'Idling':0}
df1.fillna(value=values,inplace=True)

In [10]:
printna()

Elmntkey: 52
Study_Area: 0
Sub_Area: 13180
Date Time: 0
Side: 0
Unitdesc: 0
Peak Hour_SDOT: 0
Parking_Spaces: 0
Total_Vehicle_Count: 0
Dp_Count: 0
Rpz_Count: 0
TG_Car2Go: 0
BMW_DN: 0
Lime: 0
Idling: 0
Field Notes: 151796
Construction: 13
Event Closure: 13
Subarea Label: 0
Study Year: 0
Peak Hour? (Yes or No): 0
RPZ Blocks: 78077
CSM: 77895
Time Stamp: 83564


In [47]:
#df1.to_csv('Annual_Parking_Study_Data_Cleaned.csv')

In [11]:
# The following Unitdesc's are missing Elmntkey values entirely, both in SODO area
# 6TH AVE S BETWEEN S BAYVIEW ST AND S LANDER ST (E side)
# 6TH AVE S BETWEEN S STACY ST AND S BAYVIEW ST (E side)

In [23]:
# Add some calculated columns to our dataframe

# Utilization: Total_vehicle_count/Parking_spaces
# If Parking_spaces = 0, this will come out to Inf, so it should be 1 (or 100% instead)
# Free spaces: Parking_spaces - Total_vehicle_count
# Sometimes Total vehicle count exceeds parking spaces, so it should be 0 in that case
# Some Total vehicle counts are negative and that doesn't make sense, assume typo
df1.loc[df1['Total_Vehicle_Count']<0,'Total_Vehicle_Count'] = df1.loc[df1['Total_Vehicle_Count']<0,'Total_Vehicle_Count']*(-1)
df1['Utilization'] = df1['Total_Vehicle_Count']/df1['Parking_Spaces']
df1.loc[df1['Parking_Spaces']==0,'Utilization'] = 1
df1['Free_Spaces'] = df1['Parking_Spaces'] - df1['Total_Vehicle_Count']
df1.loc[df1['Free_Spaces']<0,'Free_Spaces'] = 0

In [13]:
# Group the study areas into more coherent neighborhoods
study_areas = df1['Study_Area'].unique()
print(study_areas)

['South Lake Union' '12th Avenue' 'SODO' 'Ballard Locks'
 'Ballard Locks Summer' 'Ballard Locks summer' 'Pioneer Square'
 'Cherry Hill' 'Uptown' 'First Hill' 'Commercial Core - Retail'
 'Commercial Core - Waterfront' 'Commercial Core - Financial' 'Fremont'
 'Uptown Triangle' 'Green Lake' 'Capitol Hill' 'Westlake' 'Pike-Pine'
 'Commercial Core Retail' 'Commercial Core Financial'
 'Commercial Core Waterfront' 'Commercial Core' 'Belltown'
 'University District' 'Ballard' 'Roosevelt' 'Chinatown/ID'
 'Ballard Locks Spring' 'Pioneer Square 2017' 'Denny Triangle'
 '12th Ave 2017 Annual Study' 'Chinatown ID 2017'
 'Ballard Locks Spring 2017' 'Denny Triangle South - 2017'
 'Denny Triangle North - 2017' 'Greenlake' 'Denny Triangle South'
 'Denny Triangle North' '12th Ave - Weekday' 'Ballard - Weekday'
 'First Hill - Weekday' 'Ballard Locks - Weekday (Spring)'
 'Belltown - Weekday' 'Ballard Locks - Weekday (Summer)'
 'Capitol Hill - Weekday' 'Chinatown/ID - Event' 'Cherry Hill - Weekday'
 'Chinat

In [14]:
areas_search = ['12th Ave','15th Avenue','Ballard','Belltown',
         'Capitol Hill','Cherry Hill','Chinatown','Columbia City',
         'Commercial Core','Denny Triangle','Dexter','First Hill',
         'Fremont','Green','Lake City','Little Saigon',
         'Pike-Pine','Pioneer Square','Roosevelt','SODO',
         'South Lake Union','University District','Uptown',
         'West Seattle','Westlake']
areas = ['12th Ave','15th Ave','Ballard','Belltown',
         'Capitol Hill','Cherry Hill','Chinatown','Columbia City',
         'Commercial Core','Denny Triangle','South Lake Union','First Hill',
         'Fremont','Green Lake','Lake City','Little Saigon',
         'Pike-Pine','Pioneer Square','Roosevelt','SODO',
         'South Lake Union','University District','Uptown',
         'West Seattle','Westlake']
area_dict = {}

for key in study_areas:
    for i,nhood in enumerate(areas_search):
        if nhood in key:
            area_dict[key] = areas[i]
            break
            
df1['Neighborhood'] = df1['Study_Area'].map(area_dict)

In [15]:
print(df1['Neighborhood'].unique())

['South Lake Union' '12th Ave' 'SODO' 'Ballard' 'Pioneer Square'
 'Cherry Hill' 'Uptown' 'First Hill' 'Commercial Core' 'Fremont'
 'Green Lake' 'Capitol Hill' 'Westlake' 'Pike-Pine' 'Belltown'
 'University District' 'Roosevelt' 'Chinatown' 'Denny Triangle'
 'Little Saigon' 'Columbia City' '15th Ave' 'Lake City' 'West Seattle']


In [16]:
# make separate columns for time data since we have been having trouble parsing the datetime format
date_time = pd.to_datetime(df1['Date Time'])

obs_yr = date_time.dt.year
obs_month =date_time.dt.month
obs_wkday = date_time.dt.weekday
obs_hr = date_time.dt.hour
obs_qtr = date_time.dt.quarter
obs_daytype_dict = {0: 'weekday', 1: 'weekday', 2: 'weekday', 3: 'weekday', 4: 'weekday', 5: 'weekend', 6: 'weekend'}
obs_daytype = obs_wkday.map(obs_daytype_dict)

df1['Quarter'] = obs_qtr
df1['Month'] = obs_month
df1['Day of week'] = obs_wkday
df1['Weekday or weekend'] = obs_daytype
df1['Hour'] = obs_hr

In [21]:
df1.columns

Index(['Elmntkey', 'Neighborhood', 'Study_Area', 'Sub_Area', 'Subarea Label',
       'Date Time', 'Study Year', 'Quarter', 'Month', 'Day of week',
       'Weekday or weekend', 'Hour', 'Side', 'Unitdesc', 'Peak Hour_SDOT',
       'Parking_Spaces', 'Total_Vehicle_Count', 'Utilization', 'Free_Spaces',
       'Dp_Count', 'Rpz_Count', 'TG_Car2Go', 'BMW_DN', 'Lime', 'Idling',
       'Field Notes', 'Construction', 'Event Closure',
       'Peak Hour? (Yes or No)', 'RPZ Blocks', 'CSM', 'Time Stamp'],
      dtype='object')

In [18]:
df1 = df1[['Elmntkey', 'Neighborhood', 'Study_Area', 'Sub_Area', 'Subarea Label', 'Date Time', 'Study Year', 'Quarter', 'Month',
       'Day of week', 'Weekday or weekend', 'Hour', 'Side', 'Unitdesc',
       'Peak Hour_SDOT', 'Parking_Spaces', 'Total_Vehicle_Count', 'Utilization', 'Free_Spaces', 'Dp_Count',
       'Rpz_Count', 'TG_Car2Go', 'BMW_DN', 'Lime', 'Idling', 'Field Notes',
       'Construction', 'Event Closure',  
       'Peak Hour? (Yes or No)', 'RPZ Blocks', 'CSM', 'Time Stamp']]

In [24]:
df1.to_csv('Annual_Parking_Study_Data_Cleaned2.csv',index=False)